### **Notebook 2: Creating the Silver Layer**

#### Steps:
1. Read Bronze Delta table.
2. Clean and transform the data for the Silver layer.
3. Save the transformed data as a Delta table and register it in Unity Catalog.

#### Code:

In [ ]:
# Step 1: Read Bronze data
bronze_df = spark.read.format("delta").load("/mnt/s3data/bronze/retail_data")

# Step 2: Transform Bronze data
from pyspark.sql.functions import col
silver_df = bronze_df.filter(col("transaction_amount").isNotNull()) \
                     .withColumnRenamed("transaction_date", "date") \
                     .withColumn("year", col("date").substr(1, 4))

# Step 3: Save Silver data as a Delta table
silver_path = "/mnt/s3data/silver/retail_data"
silver_df.write.format("delta").mode("overwrite").save(silver_path)

# Step 4: Register the Silver table in Unity Catalog
spark.sql(f"""
CREATE TABLE IF NOT EXISTS retail_catalog.silver.retail_data
USING DELTA
LOCATION '{silver_path}'
""")

---